# TMA4215 - Assignment 2 - Conjugate Gradient and ODEs

**Deadline:** Wednesday September 23, 11:59PM.


In this assignment you will implement the Preconditioned Conjugate Gradient (PCG) method.
Then there are some small theoretical questions on Numerical ODEs.

**PCG algorithm.** Implementations of the (P)CG algorithm can be found in many libraries, but here you are supposed to implement your own code for solving $Ax=b$ by PCG. Here we shall assume that the matrix $A$ is implemented by means of a function, say *A_func(v)* that for any vector $v$ returns $Av$. Similarly, we implement the preconditioner also as a function *Precond(v)* that returns $P^{-1}v$ for any vector $v$. An popular alternative to this approach is to use sparse representation of matrices, (see e.g. *scipy.sparse*), but we shall not use that here.

You are of course allowed to use the obvious built-in functions for elementary operations, such as computing inner products, norms etc. We will provide the function *A_func(v)* to be used in this assignment. Your job is to implement a function *PCG* whose template you find below, a function *Precond(v)* that implements some iterations of the Gauss-Jacobi splitting method as preconditioner, and finally a small main programme that generates the results.

**Problem 1.** 

**(a)** Fill in the templates below for a function *PCG* 

*PCG* should take 5 input arguments, 
1. the function *A_func* that takes $v$ as input and returns $A\cdot v$.
2. the right hand side *b* of the system $A\cdot x = b$ (numpy array)
3. The initial guess *x0*
4. *tol*, a tolerance used to define the stopping criterion
5. *Precond* which is a function that takes a vector $v$ as input and returns $P^{-1}v$ for the preconditioner $P$.

There should be 3 output arguments: 
1. the computed solution $x$, 
2. the residual after the last iteration, $r$ 
3. the number of iterations *iter* needed to meet the stopping criterion 

The stopping criterion can be taken as ($\|r\|<$*tol* or *iter*$\geq N$) where $N$ is the dimension of the problem.





**(b)** Write a Gauss-Jacobi preconditioner function (see below *PrecJac*)
Test your implementation using the supplied function *A_func* which takes as input a vector $v\in\mathbb{R}^{n^2}$ and returns the vector in $A\cdot v\in\mathbb{R}^{n^2}$. This corresponds to a discretisation of the Poisson equation

$$ -\nabla^2 u = f $$

in the plane, so that there are $n$ discretisation points in each direction, thereby dimension $n^2$ of the unknown solution vector. You can take *x0* to be the 0-vector in $\mathbb{R}^{n^2}$. The right hand side is also obtained by a supplied function *b_def* (see below) that from an input integer $n$ returns a vector $b\in\mathbb{R}^{n^2}$.

*PrecJac(v)*. This function can be considered as tailored for the example given by *A_func*.
Starting with the 0-vector as initial guess, apply a specified number of iterations with the Gauss-Jacobi iterative method to the problem $Au=v$. The number of iterations can be specified in the main program. 
You need the diagonal of $A$ to construct the preconditioner and you can assume that $M=\text{diag}(A)=4I$ (four times the identity matrix). Therefore, via $A=M-N$ in Gauss-Jacobi, you can write $N=M-A=4I-A$ so that the GJ-iteration becomes

$$
     u^{(k+1)} = M^{-1}N u^{k} + M^{-1}b = (I-\frac14 A)u^{(k)}+\frac 14 b = u^{(k)}+\frac14 (b-Au^{(k)}),\quad
     u^{(0)} = 0
$$

Try to run the program with and without preconditioner for $N=n^2, \ n=20, 40, 60, \ldots, 200$. For each $n$, print out a line with $n$, *iterNull* and *iterJac2*, and *iterJac4*. Here *iterNull* is without preconditoning,
*iterJac2* and *iterJac4* is with 2 and 4 Gauss-Jacobi iterations respectively.

You can use *tol*$=10^{-5}$ in all experiments. Always let *x0* be the zero-vector.

**Control question 1.** How many CG iterations are needed with no preconditioning, $N=n^2=256 (n=16)$ with *tol* and *x0* as specified above.

**Control question 2.** How many PCG iterations are needed with preconditioning, 4 Gauss-Jacobi iterations, $N=n^2=256 (n=16)$ with *tol* and *x0* as specified above.

In [114]:
import numpy as np
import numpy.linalg as la


def PCG(A_func,b,x0,tol,Precond):
    N=len(x0)
    iter = 0
    
    x = x0
    r = b - A_func(x)
    d = Precond(r)
    while (la.norm(r) >= tol and iter < N):
        alphaK = (r.T@Precond(r))/(d.T@A_func(d))
        x = x + alphaK*d
        rnew = r - alphaK*A_func(d)       
        betaK = (rnew.T@Precond(rnew))/(r.T@Precond(r))
        d = Precond(rnew) + betaK*d
        r = rnew
        iter = iter + 1
    return x, iter, r

def PCG_wiki(A_func,b,x0,tol,Precond):
    N=len(x0)
    iter = 0
    x = x0
    
    r = b - A_func(x)
    d = Precond(r)
    delta_new = r.T*d
    delta_0 = delta_new
    while iter < N and la.norm(delta_new) > tol*tol*la.norm(delta_0):
        q = A_func(d)
        alpha = delta_new/(d.T*q)
        x = x + alpha*d
        if iter%50==0:
            r = b-A_func(x)
        else:
            r = r - alpha*q
        s = Precond(r)
        delta_old, delta_new = delta_new, r.T*s
        beta = delta_new/delta_old
        d = s + beta*d
        iter = iter+1
    return x, iter, delta_new

def PrecNull(r):
    return r

def PrecJac(v):
    global JACITER
    x0 = np.zeros_like(v) #?
    u = x0
    for k in range(JACITER):
        u = u + 1/4 * (v - A_func(u))
    return u # which equals P^{-1}v


In [116]:
# Supplied functions you can use for Problem 1b.
import sys
from prettytable import PrettyTable


def A_func(v):
# Note that v of length N must be square, N=n**2 for some n.
    n2=len(v)
    n=int(np.sqrt(n2))
    if n**2 != n2:
        sys.exit("Size of vector must be a square")
    M=np.zeros((n+2,n+2))
    M[1:n+1,1:n+1]=np.reshape(v,(n,n))
    AM = 4*M[1:n+1,1:n+1]-M[:n,1:n+1]-M[2:n+2,1:n+1]-M[1:n+1,:n]-M[1:n+1,2:n+2]
    Av = np.reshape(AM,(n2,))
    return Av

def b_def(n):
    h = 1/(n+1)
    X = np.linspace(h,1-h,n)
    Y=X
    B = np.outer( 16*X**2*(1-X)**2, 16*Y**2*(1-Y)**2 )
    b = np.reshape(B, (n**2,))
    return B,b


#MAIN
nLst = [n for n in range(0,200+1,20)]
nLst[0] = 16
tol=1e-5
prT = PrettyTable() #for printing
prT.field_names = ["n", "iterNull", "iterJac2", "iterJac4"]
for n in nLst:
    #we overwrite x and r but we dont care for the table
    x0 = np.zeros(n*n) #shape n^2
    b = b_def(n)[1] #i assume i want the second returned arg?
    x, iterNull, r = PCG(A_func, b, x0, tol, PrecNull)
    JACITER = 2
    x, iterJac2, r = PCG(A_func, b, x0, tol, PrecJac)
    JACITER = 4
    x, iterJac4, r = PCG(A_func, b, x0, tol, PrecJac)
    prT.add_row([n, iterNull, iterJac2, iterJac4])
print(prT)

+-----+----------+----------+----------+
|  n  | iterNull | iterJac2 | iterJac4 |
+-----+----------+----------+----------+
|  16 |    21    |    12    |    9     |
|  20 |    26    |    14    |    11    |
|  40 |    53    |    28    |    20    |
|  60 |    81    |    42    |    30    |
|  80 |   109    |    56    |    40    |
| 100 |   137    |    70    |    50    |
| 120 |   168    |    84    |    60    |
| 140 |   198    |   100    |    70    |
| 160 |   226    |   115    |    81    |
| 180 |   255    |   129    |    92    |
| 200 |   283    |   143    |   102    |
+-----+----------+----------+----------+


**Problem 2.** 

**(a)** Find the stability function of the Runge-Kutta method whose Butcher tableaux is given as

$$
\begin{array}{c|rr}
\frac12 & \frac12 & 0 \\
\frac32 & -\frac12 & 2 \\ \hline
        & -\frac12 & \frac32
\end{array}
$$

**(b)** Does the stability region of this method contain the left half plane, $\mathbb{C}^-=\{z\in\mathbb{C}: \text{Re}(z)<0\}$?

**Control question 3.** A multiple choice question for the stability function.

**Control question 4.** A yes/no answer for **(2b)**



**(a)** By reading the Butcher table, we notice that it is a two-step implicit Runge Kutta method given by

$$
K_1 = f\left(t_n + \frac12 h, u_n + \frac12 h K_1\right) \\
K_2 = f\left(t_n + \frac32 h, u_n + h\left(\frac{-1}{2}K_1+2 K_2\right)\right) \\
u_{n+1} = u_n + h\left(\frac{-1}{2} K_1 + \frac32 K_2\right)
$$

By applying the method to the test equation $y'=\lambda y$, we should get $u_{n+1} = R(h\lambda) u_n$:

$$
\implies K_1 = f(t_2+1/2 h, \lambda y + 1/2 h K_1) = \lambda (\lambda y + 1/2 h K_1) \\
\implies K_1-0.5hk_1=K_1(1-0.5h)=\lambda^2 y
$$

$$\implies K_1 = \lambda(u_n + \frac12 hK_1) \implies K_1(1-\frac12\lambda h)=\lambda u_n \\
K_1 = \frac{2\lambda u_n}{2-h\lambda}
K_2 = \lambda(u_n + h(\frac{-1}{2}K_1+2K_2))\\
K_2(1-\lambda 2h)=\lambda(u_n + \frac{-1}{2}hK_1) \\
\implies K_2 = \dots = \frac{2\lambda u_n}{2-h\lambda}$$

$$
\implies u_{n+1} = u_n - \frac{h\lambda u_n}{2-h\lambda}+\frac{3h\lambda u_n}{2-h\lambda}=u_n+\frac{2h\lambda u_n}{2-h\lambda} = \frac{2+h\lambda}{2-h\lambda} u_n = R(h\lambda) u_n \\
\\
\therefore R(z)=\frac{2+z}{2-z}
$$

**This is apparently wrong according to control questions, something I don't understand.. Oh well!**

**(b)**
The method's stability region is $S:=\{z\in \mathbb{C} : |R(z)| < 1 \}$ and claim is $ \mathbb{C}^- \subseteq S$.

Claim:
$R(z)=\frac{|2+z|}{|2-z|} < 1 \forall \{z \in \mathbb{C} : \Re(z) < 0\}$
If we show that the set of solutions to the problem 
$$\left\{z\in\mathbb{C} : \frac{|2+z|}{|2-z|} \ge 1 \wedge \Re(z)< 0 \right\} = \emptyset, $$
The claim would have been proven. This is possible to prove, however I was not able to.